# Generate Splits
* Create training, validation and test splits

In [ ]:
import math
import os

import numpy as np
import pandas as pd
import yaml
from tqdm import tqdm

In [ ]:
source_dir = "../../data/processed_data"

In [ ]:
outdir = "../../data/splits"
os.makedirs(outdir, exist_ok=True)

In [ ]:
media = ""

# Sort items by timestamp

In [ ]:
def shard_by_user(file, num_shards):
    try:
        outfiles = []
        for i in range(num_shards):
            outfiles.append(open(f"{file}.shard.{i}", "w"))
        with open(file) as in_file:
            header = False
            for line in tqdm(in_file):
                if not header:
                    header = True
                    user_col = line.strip().split(",").index("userid")
                    for f in outfiles:
                        f.write(line)
                    continue
                fields = line.strip().split(",")
                user = fields[user_col]
                outfiles[int(user) % num_shards].write(line)
    finally:
        for f in outfiles:
            f.close()


def temporal_sort(input_fn, output_fn):
    df = pd.read_csv(input_fn, dtype=str)
    for key, dtype in zip(["userid", "update_order", "updated_at"], [int, int, float]):
        df[key] = df[key].astype(dtype)
    df = df.sort_values(by=["userid", "update_order", "updated_at"]).reset_index(
        drop=True
    )
    df["unit"] = 1
    df["forward_order"] = (
        df.groupby("userid", group_keys=False)["unit"]
        .apply(lambda x: x.cumsum())
        .values
    )
    df["backward_order"] = (
        df.groupby("userid", group_keys=False)["unit"]
        .apply(lambda x: x.cumsum()[::-1])
        .values
    )
    df.to_csv(output_fn, index=False)


def sharded_temporal_sort(input_fn, output_fn, num_shards=16):
    shard_by_user(input_fn, num_shards)
    for i in tqdm(range(num_shards)):
        temporal_sort(f"{input_fn}.shard.{i}", f"{output_fn}.shard.{i}")
        os.remove(f"{input_fn}.shard.{i}")
    with open(output_fn, "w") as outfile:
        for i in tqdm(range(num_shards)):
            fn = f"{output_fn}.shard.{i}"
            with open(fn) as infile:
                header = False
                for line in infile:
                    if not header:
                        header = True
                        if i == 0:
                            outfile.write(line)
                        continue
                    outfile.write(line)
            os.remove(fn)

In [ ]:
sharded_temporal_sort(
    os.path.join(source_dir, f"user_{media}_list.csv"),
    os.path.join(outdir, f"user_{media}_list.sorted.training.csv"),
)

In [ ]:
sharded_temporal_sort(
    os.path.join(source_dir, f"prune.{media}.knowledge_cutoff.csv"),
    os.path.join(outdir, f"user_{media}_list.sorted.test.csv"),
)

# Generate splits!

In [ ]:
def get_split(userid):
    if userid % 100 == 0:
        return "validation"
    else:
        return "training"

In [ ]:
def get_cutoff(days):
    def parse_line(file, field, format=int):
        line = file.readline()
        fields = line.strip().split(",")
        assert len(fields) == 2
        assert fields[0] == field
        return format(fields[1])

    with open(os.path.join(source_dir, "timestamps.csv")) as f:
        min_timestamp = parse_line(f, "min_timestamp")
        max_timestamp = parse_line(f, "max_timestamp")

    with open(os.path.join(source_dir, "knowledge_cutoff.csv")) as f:
        knowledge_cutoff = parse_line(f, "knowledge_cutoff", float)

    seconds_in_day = 24 * 60 * 60
    return knowledge_cutoff - days * seconds_in_day / (max_timestamp - min_timestamp)

In [ ]:
def generate_training_splits(timestamp_cutoff, num_interactions):
    input_fn = os.path.join(outdir, f"user_{media}_list.sorted.training.csv")
    files = {
        x: open(os.path.join(outdir, f"{x}.user_{media}_list.csv"), "w")
        for x in ["training", "validation"]
    }
    with open(input_fn) as f:
        header = False
        for line in tqdm(f):
            fields = line.strip().split(",")
            if not header:
                header = True
                timestamp_col = fields.index("updated_at")
                user_col = fields.index("userid")
                order_col = fields.index("backward_order")
                for g in files.values():
                    g.write(line)
                continue
            userid = int(fields[user_col])
            timestamp = float(fields[timestamp_col])
            order = int(fields[order_col])
            train = (
                (timestamp < timestamp_cutoff)
                or (order > num_interactions)
                or get_split(userid) == "training"
            )
            split = "training" if train else "validation"
            files[split].write(line)
    for f in files.values():
        f.close()
    os.remove(input_fn)

In [ ]:
def generate_test_splits(num_interactions):
    input_fn = os.path.join(outdir, f"user_{media}_list.sorted.test.csv")
    output_fn = os.path.join(outdir, f"test.user_{media}_list.csv")
    with open(input_fn) as f:
        with open(output_fn, "w") as g:
            header = False
            for line in tqdm(f):
                fields = line.strip().split(",")
                if not header:
                    header = True
                    user_col = fields.index("userid")
                    order_col = fields.index("forward_order")
                    g.write(line)
                    continue
                userid = int(fields[user_col])
                order = int(fields[order_col])
                if order <= num_interactions and get_split(userid) == "training":
                    g.write(line)
    os.remove(input_fn)

In [ ]:
def get_settings():
    with open("../../environment/settings.yml", "r") as f:
        return yaml.safe_load(f)

In [ ]:
# interactions will go in the val split if the user is in the val split
# AND the the interaction is one of the user's N most recent watches
# AND the interaction occured less that M days ago
settings = get_settings()

In [ ]:
generate_training_splits(
    get_cutoff(settings["cutoff_days"]),
    settings["cutoff_interactions"],
)

In [ ]:
generate_test_splits(num_interactions)